-for all categories except lectures, create sentiment score for each sentence, linear regression using interjection sandwiched by comments, predict comments change in sentiment score using the text of interjection

In [1]:
%pylab inline
import re
import os
import pandas as pd
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
from sklearn.base import TransformerMixin
from sklearn.pipeline import Pipeline
import pandas as pd

Populating the interactive namespace from numpy and matplotlib


In [2]:
contents = []
directory = '/Users/henryraymond/Documents/jupyter/NLP_fall22/final/TRN/'

for filename in os.listdir(directory):
    f = os.path.join(directory, filename)    
    filehandle = open(f, 'r', encoding="ISO-8859-1")
    text = ''
    while True:
        line = filehandle.readline()
        if not line:
            break
        text += line
        
    contents.append(text)
    filehandle.close()
    
dic = {'contents' : contents}

In [3]:
categories = pd.read_excel('/Users/henryraymond/Documents/jupyter/NLP_fall22/final/transcript_categories.xlsx')
dic['category'] = categories.category

In [4]:
df = pd.DataFrame.from_dict(dic)

In [5]:
import spacy
import string
from spacy.lang.en.stop_words import STOP_WORDS
from spacy.lang.en import English

# Create our list of punctuation marks
punctuations = string.punctuation

# Create our list of stopwords
nlp = spacy.load('en_core_web_sm')
stop_words = spacy.lang.en.stop_words.STOP_WORDS

# Load English tokenizer, tagger, parser, NER and word vectors
parser = English()

# Creating our tokenizer function
def spacy_tokenizer(sentence):
    # Creating our token object, which is used to create documents with linguistic annotations.
    mytokens = nlp(sentence)

    # Lemmatizing each token and converting each token into lowercase
    mytokens = [ word.lemma_.lower().strip() if word.lemma_ != "-PRON-" else word.lower_ for word in mytokens ]

    # Removing stop words
    mytokens = [ word for word in mytokens if word not in stop_words and word not in punctuations ]

    # return preprocessed list of tokens
    return mytokens

In [6]:
# Custom transformer using spaCy
class predictors(TransformerMixin):
    def transform(self, X, **transform_params):
        # Cleaning Text
        return [clean_text(text) for text in X]

    def fit(self, X, y=None, **fit_params):
        return self

    def get_params(self, deep=True):
        return {}

# Basic function to clean the text
def clean_text(text):
    # Removing spaces and converting text into lowercase
    return text.strip().lower()

In [7]:
bow_vector = CountVectorizer(tokenizer = spacy_tokenizer, ngram_range=(1,1))
tfidf_vector = TfidfVectorizer(tokenizer = spacy_tokenizer)

In [8]:
df.columns

Index(['contents', 'category'], dtype='object')

In [9]:
from sklearn.model_selection import train_test_split

X = df['contents'] # the features we want to analyze
ylabels = df['category'] # the labels, or answers, we want to test against

X_train, X_test, y_train, y_test = train_test_split(X, ylabels, test_size=0.3)

In [10]:
%%time
# Logistic Regression Classifier
from sklearn.linear_model import LogisticRegression
classifier = LogisticRegression(max_iter=10000)

# Create pipeline using Bag of Words
pipe = Pipeline([("cleaner", predictors()),
                 ('vectorizer', tfidf_vector),   # replace with tf_idf, or bow to try
                 ('classifier', classifier)])

# model generation
pipe.fit(X_train,y_train)

CPU times: user 1min 18s, sys: 11.3 s, total: 1min 29s
Wall time: 1min 29s


Pipeline(steps=[('cleaner', <__main__.predictors object at 0x7f7ea785f550>),
                ('vectorizer',
                 TfidfVectorizer(tokenizer=<function spacy_tokenizer at 0x7f7ea328e280>)),
                ('classifier', LogisticRegression(max_iter=10000))])

In [11]:
from sklearn import metrics
# Predicting with a test dataset
predicted = pipe.predict(X_test)

# Model Accuracy
print("Logistic Regression Accuracy:",metrics.accuracy_score(y_test, predicted))
print("Logistic Regression Precision:",metrics.precision_score(y_test, predicted, average=None))
print("Logistic Regression Recall:",metrics.recall_score(y_test, predicted, average=None))

Logistic Regression Accuracy: 0.6666666666666666
Logistic Regression Precision: [0.66666667 0.         0.66666667 0.        ]
Logistic Regression Recall: [0.90909091 0.         0.5        0.        ]


/Users/henryraymond/opt/anaconda3/lib/python3.8/site-packages/sklearn/metrics/_classification.py:1245: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
